In [1]:
# Imports
import sys, os, math, pickle
import random as rn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tqdm import tqdm #To show the progress

# Change Pandas dislpay option
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')

In [2]:
def prediction(input_x, model, sc_x, sc_y):
    x_sc = sc_x.transform(input_x)
    y_pred_sc = model.predict(x_sc, batch_size=4096)
#     y_pred_sc = model.predict(x_sc, batch_size=4096, verbose=True)
    y_pred = sc_y.inverse_transform(y_pred_sc)
    return y_pred

In [3]:
# Load AI model
model_fff = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/FFF_Flow/FFF_Flow.h5')
model_thermaleff = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/ThermalEff/ThermalEff.h5')
model_specpower = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/SpecPower/Output/SpecPower.h5')
model_mche_out_t = load_model('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/MCHE_OUT_T/MCHE_OUT_T.h5')

# Load Scaler used for Model
import pickle
sc_x_fff = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/FFF_Flow/FFF_Flow_sc_x.pkl', "rb"))
sc_y_fff= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/FFF_Flow/FFF_Flow_sc_y.pkl',"rb"))
sc_x_thermaleff = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/ThermalEff/ThermalEff_sc_x.pkl',"rb"))
sc_y_thermaleff = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/ThermalEff/ThermalEff_sc_y.pkl',"rb"))
sc_x_specpower = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/SpecPower/Output/SpecPower_sc_x.pkl',"rb"))
sc_y_specpower = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/SpecPower/Output/SpecPower_sc_y.pkl',"rb"))
sc_x_mche_out_t = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/MCHE_OUT_T/MCHE_OUT_T_sc_x.pkl',"rb"))
sc_y_mche_out_t = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step1_Regression/MCHE_OUT_T/MCHE_OUT_T_sc_y.pkl',"rb"))

In [4]:
# Count File Numbers
data_dir = '/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Dataset'
data_filelist = os.listdir(data_dir)
file_num = len(data_filelist)
print("file_num:",file_num)

# Read Data and Integrate into 1 Data Frame
df_raw = pd.DataFrame()
for i_file in range(file_num):
    file_dir = data_dir + "/" + data_filelist[i_file]
    df_tmp = pd.read_csv(file_dir)
    df_raw = pd.concat([df_raw, df_tmp])
df_raw.reset_index(drop=True, inplace=True)
print("df_raw.shape:", df_raw.shape)

# remove Note, Unnamed columns and Feasible=0.
df = df_raw.copy()
df.drop(["Note", "Unnamed: 0"], axis = 1, inplace = True)
df = df[df['Feasible'] != 0]
print("df.shape:", df.shape)
df.describe()

file_num: 1
df_raw.shape: (96431, 37)
df.shape: (78492, 35)


,FEED_N2,FEED_C1,FEED_C2,FEED_C3,FEED_iC4,FEED_nC4,FEED_C5+,AmbTemp,C3_CON_OUT,P_HPMR,P_LPMR,MCHE_BTM_DT,WB_UA,CB_UA,MR_N2,MR_C1,MR_C2,MR_C3,LMR_GASFLOW,FFF_Flow,SpecPower,ThermalEff,MCHE_OUT_T,MCHE_OUT_C_T,MCHE_MID_T,MCHE_MID_C_T,MCHE_IN_T,MCHE_IN_C_T,MR_GASFLOW,HMR_VFLOW,C3_GT_Power,LPMR_GT_Power,HPMR_GT_Power,LNG_Dens,Feasible
count,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.0,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,7.849200e+04,7.849200e+04,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.000000,78492.0
mean,0.772967,92.597098,3.694397,1.775303,0.580118,0.580118,0.0,28.561713,51.796812,54.578231,1.935224,4.310382,1.544166e+07,1.902363e+06,6.105164,41.152441,41.444184,11.298211,137475.567754,20.008117,320.490911,0.898645,-151.004569,-159.303782,-131.857253,-134.284109,-32.434272,-36.102467,574471.007652,937.079677,32328.622646,32670.171108,28039.295677,442.979523,1.0
std,0.166580,0.277516,0.218594,0.166677,0.120534,0.120534,0.0,4.682627,4.836847,2.967038,0.144139,2.010944,9.387198e+05,3.000158e+05,0.591600,1.249483,1.597669,1.545636,37257.688671,9804.950969,17.875592,0.020135,4.176413,2.159303,6.287577,4.489799,2.192407,2.995890,37713.135708,24.194081,2516.566780,2647.127067,2312.634130,6.449452,0.0
min,0.500000,92.000000,3.300000,1.500000,0.350000,0.350000,0.0,20.000000,41.200000,50.000000,1.700000,1.000000,1.147112e+07,9.885195e+05,5.000000,38.000000,38.000000,8.000000,48412.644417,-51230.352279,277.350263,0.763167,-158.450000,-164.102375,-143.800000,-144.250753,-36.616375,-43.477822,465391.775106,825.689419,25596.537927,24926.451594,20268.326485,406.035979,1.0
25%,0.600000,92.400000,3.500000,1.600000,0.500000,0.500000,0.0,24.500000,47.700000,52.000000,1.820000,2.500000,1.502264e+07,1.774307e+06,5.600000,40.300000,40.200000,10.100000,109171.741976,-4867.963360,306.648624,0.896757,-153.850000,-160.810486,-136.400000,-137.510426,-34.308042,-38.244614,546714.041059,921.125299,30395.051166,30799.261690,26481.384248,440.052791,1.0
50%,0.800000,92.600000,3.700000,1.800000,0.600000,0.600000,0.0,29.000000,52.200000,54.500000,1.940000,4.500000,1.546086e+07,1.894295e+06,6.200000,41.300000,41.600000,11.400000,133104.271920,2291.899625,319.993507,0.905509,-152.050000,-159.515941,-133.200000,-135.023473,-32.263562,-36.078016,572672.251434,937.544482,32349.624738,32552.585713,28055.024633,444.269590,1.0
75%,0.900000,92.800000,3.900000,1.900000,0.700000,0.700000,0.0,32.500000,55.700000,57.000000,2.060000,6.000000,1.583172e+07,2.196960e+06,6.600000,42.200000,42.800000,12.500000,161874.523630,6589.746923,333.151365,0.909341,-149.050000,-158.092988,-128.600000,-131.799195,-30.664594,-33.936834,600231.205762,953.938914,34216.553640,34430.544650,29622.096293,447.258779,1.0
max,1.000000,93.000000,4.000000,2.000000,0.750000,0.750000,0.0,36.000000,61.200000,60.000000,2.180000,7.500000,2.128458e+07,2.946253e+06,7.000000,43.000000,44.000000,14.000000,304559.624205,20473.859216,388.964464,0.917289,-128.850000,-149.084936,-101.700000,-112.936997,-28.136365,-28.489137,730102.976212,1009.998008,40615.931837,43681.622831,35843.886623,456.175112,1.0


In [5]:
# explanatory variable - given
list_u = ['FEED_N2', 'FEED_C1', 'FEED_C2','FEED_C3', #'FEED_iC4', 'FEED_nC4','FEED_C5+',
    'AmbTemp', 'C3_CON_OUT', # 'MCHE_IN_T',
    'WB_UA', 'CB_UA', #"Sub_Out_Temp_AFC","Cooler_Out_Temp_AFC",
         ]

# explanatory variable - search
list_v = ['P_HPMR', 'P_LPMR', 'MCHE_BTM_DT', 
          #'MR_N2',
          'MR_C1', 'MR_C2', 'MR_C3', #'MR_C4'
         ]

# For Result
list_result = list_v + ["FFF_Flow_pred",'ThermalEff_pred',
                        'SpecPower_pred', "MCHE_CB_Out_Temp_pred"
                       ]
# explanatory variable
list_x = list_u + list_v 

# Fix UA with Center Value
df[['WB_UA', 'CB_UA']] = df[['WB_UA', 'CB_UA']].quantile(0.5).values

# Prepare df
df_u = df.loc[:, list_u].copy().drop_duplicates()
df_v = df.loc[:, list_v].copy().drop_duplicates()
df_x = df.loc[:, list_x].copy().drop_duplicates()

In [6]:
print("Number of combinations of operatable conditions (list_v - search):", df_v.shape[0])

Number of combinations of operatable conditions (list_v - search): 78484


In [7]:
# Number of Total Row . External Conditions Qty.
df_training = df_u.copy()
print("Number of combinations of external conditions (list_u - given):", df_training.shape[0])

Number of combinations of external conditions (list_u - given): 73259


In [ ]:
%%time
output_file_num = 1 #Separation of df_training
df_total_row = df_training.shape[0]//output_file_num
print("df_total_row_per_1_file:", df_total_row)

# Optimum Parameter Search
for j in range(output_file_num):

    for row, u in tqdm(df_training[j*df_total_row:(j+1)*df_total_row].iterrows()):
        df_search = df_v.copy()
        for col in list_u:
            df_search[col] = u[col]
        df_search = df_search.loc[:, list_x]

        input_x = df_search[list_x].values

        df_search["FFF_Flow_pred"] = prediction(input_x, model_fff, sc_x_fff, sc_y_fff)
        df_search["ThermalEff_pred"] = prediction(input_x, model_thermaleff, sc_x_thermaleff, sc_y_thermaleff)
        df_search["SpecPower_pred"] = prediction(input_x, model_specpower, sc_x_specpower, sc_y_specpower)
        df_search["MCHE_CB_Out_Temp_pred"] = prediction(input_x, model_mche_out_t, sc_x_mche_out_t, sc_y_mche_out_t)
        df_search = df_search[df_search["FFF_Flow_pred"] > 0]
        max_thermal = df_search["ThermalEff_pred"].max()
        rs_best = df_search[df_search["ThermalEff_pred"] == max_thermal]
        rs_best1 = df_search[df_search["ThermalEff_pred"] > max_thermal * 0.9999]
        rs_best2 = df_search[df_search["ThermalEff_pred"] > max_thermal * 0.9998]

        # Number of extracted row
        df_training.loc[row, 'Best1_Count'] = rs_best1.shape[0]
        df_training.loc[row, 'Best2_Count'] = rs_best2.shape[0]

        # write results
        for col in list_result:
            df_training.loc[row, 'Best_' + col + '_MEAN'] = rs_best[col].mean()

            df_training.loc[row, 'Best1_' + col + '_MIN'] = rs_best1[col].min()
            df_training.loc[row, 'Best1_' + col + '_MAX'] = rs_best1[col].max()
            df_training.loc[row, 'Best1_' + col + '_MEAN'] = rs_best1[col].mean()
            df_training.loc[row, 'Best1_' + col + '_STD'] = rs_best1[col].std()
            
            df_training.loc[row, 'Best2_' + col + '_MIN'] = rs_best2[col].min()
            df_training.loc[row, 'Best2_' + col + '_MAX'] = rs_best2[col].max()
            df_training.loc[row, 'Best2_' + col + '_MEAN'] = rs_best2[col].mean()
            df_training.loc[row, 'Best2_' + col + '_STD'] = rs_best2[col].std()

    df_training[j*df_total_row:(j+1)*df_total_row].to_pickle('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step2_OptimumParameter/output_pkl/searched_{}.pkl'.format(j))
    df_training[j*df_total_row:(j+1)*df_total_row].to_csv('/content/drive/MyDrive/Colab Notebooks/CGH/AID2/Step2_OptimumParameter/output_csv/searched_{}.csv'.format(j))

0it [00:00, ?it/s]

df_total_row_per_1_file: 73259


60374it [6:41:30,  2.81it/s]